In [ ]:
import boto3
from data import data
from clean_harvest_report import convert_mixed_buckets_to_clean_buckets
import os
import json

s3 = boto3.resource('s3')
bucket = 'aquabyte-images-adhoc'
s3_bucket = s3.Bucket(bucket)
s3_dir = 'alok/production_datasets/'

for ds in data:
    path = os.path.join(s3_dir, ds, 'ground_truth_metadata_buckets.json')
    obj = s3.Object(bucket, path)
    meta = json.loads(obj.get()['Body'].read().decode('utf-8'))
    if meta['raw_buckets']:
        meta['clean_buckets'] = convert_mixed_buckets_to_clean_buckets(
            meta['raw_buckets'], meta['expected_loss_factor'])
    with open('clean_buckets.json', 'w') as f:
        f.write(json.dumps(meta))
    s3.meta.client.upload_file('clean_superior_buckets.json', bucket, path)